<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Langgraph - Parallel Evaluation

Parallel Evaluation in LangGraph
In this tutorial, we’ll build a parallel execution workflow using LangGraph — ideal for scenarios where multiple evaluations or subtasks can run independently before being aggregated into a final decision.

Our application generates a compelling product description and then runs three checks in parallel:

- Safety Check: Is the content safe and non-violent?

- Policy Compliance: Does it follow company policy?

- Clarity Check: Is it understandable to a general audience?

This pattern demonstrates how to fan out execution after a shared generation step, and aggregate results before producing a final output.

We use Phoenix tracing to gain full visibility into each node execution, making it easy to debug or audit how decisions were made across the parallel branches.

In [ ]:
!pip install langgraph langchain langchain_community "arize-phoenix" arize-phoenix-otel openinference-instrumentation-langchain

In [ ]:
import getpass
import os

from langgraph.graph import END, START, StateGraph

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Configure Phoenix Tracing

Make sure you go to https://app.phoenix.arize.com/ and generate an API key. This will allow you to trace your Langgraph application with Phoenix.

In [ ]:
PHOENIX_API_KEY = getpass.getpass("Phoenix API Key:")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="Parallel", auto_instrument=True)

In [ ]:
from langchain.chat_models import ChatOpenAI
from typing_extensions import TypedDict

# LLM of choice

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# Graph State Definition
We define a State object to keep track of all data flowing through our LangGraph. This includes the input product name, the generated description, results of three independent evaluation checks, and the final aggregated output.

In [ ]:
class State(TypedDict):
    product: str
    description: str
    safety_check: str
    policy_check: str
    clarity_check: str
    final_output: str

# Node 1: Generate Product Description
This node uses the LLM to write a compelling marketing-style description of the product. The output is stored in the description field of the graph state.

In [ ]:
def generate_description(state: State):
    msg = llm.invoke(f"Write a compelling product description for: {state['product']}")
    return {"description": msg.content}

# Node 2–4: Parallel Evaluation Checks
After the product description is created, we fan out to three evaluators, each performing an independent check in parallel:

**Safety Check**: Is the language safe and non-violent?

**Policy Compliance**: Does it align with company guidelines?

**Clarity Check**: Is it understandable by a general audience?

Each function receives the same description as input and returns a binary decision ("yes" or "no").

In [ ]:
def check_safety(state: State):
    msg = llm.invoke(
        f"Is this product description safe and non-violent? Answer yes or no.\n\n{state['description']}"
    )
    return {"safety_check": msg.content}


def check_policy(state: State):
    msg = llm.invoke(
        f"Does this product description comply with our company policy? Answer yes or no.\n\n{state['description']}"
    )
    return {"policy_check": msg.content}


def check_clarity(state: State):
    msg = llm.invoke(
        f"Is this description clear and understandable to a 10th-grade reader? Answer yes or no.\n\n{state['description']}"
    )
    return {"clarity_check": msg.content}

# Node 5: Aggregate the Results
Once the checks complete, this node gathers their responses. If all checks return "yes", the product description is approved. Otherwise, it’s flagged as rejected, along with reasons.

In [ ]:
def aggregate_results(state: State):
    if (
        "yes" in state["safety_check"].strip().lower()
        and "yes" in state["policy_check"].strip().lower()
        and "yes" in state["clarity_check"].strip().lower()
    ):
        return {"final_output": state["description"]}
    return {
        "final_output": "REJECTED: One or more checks failed.\n"
        f"Safety: {state['safety_check']}, Policy: {state['policy_check']}, Clarity: {state['clarity_check']}"
    }

# Building the Parallel Evaluation Graph
With all our nodes defined, we now assemble them into a LangGraph using StateGraph.

**Start → Description**: We begin by generating the product description.

**Fan Out Checks**: The output fans out into three parallel paths — safety, policy, and clarity checks — enabling efficient, simultaneous validation.

**Converge → Aggregate**: Once all checks complete, the results converge into a final aggregation node that determines whether to approve or reject the description.

**End**: The final result is produced.

This setup showcases LangGraph’s ability to manage parallelism and convergence, streamlining complex workflows while remaining transparent and modular.

In [ ]:
builder = StateGraph(State)

builder.add_node("generate_description", generate_description)
builder.add_node("check_safety", check_safety)
builder.add_node("check_policy", check_policy)
builder.add_node("check_clarity", check_clarity)
builder.add_node("aggregate_results", aggregate_results)

# Description generation first
builder.add_edge(START, "generate_description")

# Then fan out for parallel checks
builder.add_edge("generate_description", "check_safety")
builder.add_edge("generate_description", "check_policy")
builder.add_edge("generate_description", "check_clarity")

# All checks go to the aggregator
builder.add_edge("check_safety", "aggregate_results")
builder.add_edge("check_policy", "aggregate_results")
builder.add_edge("check_clarity", "aggregate_results")

# Final result
builder.add_edge("aggregate_results", END)

workflow = builder.compile()

# Example Usage

In [ ]:
state = workflow.invoke({"product": "Smart glasses that project your calendar"})
print(state["final_output"])

# Make sure to view your traces in Phoenix!